# Generate html files to facilitate result inspection
This notebook can generate the html files to fascilitate the inspection of the visualiation elements.
Finish the following to do items before using this notebook:

1. Download and organize the datafilea using [this example jupyter notebook](https://github.com/xiyuyi-at-LLNL/llsmvis/blob/main/Download-and-inspect-analysis-results.ipynb)
2. Clone or download the llsmvis repository from [the llsmvis github page](https://github.com/xiyuyi-at-LLNL/llsmvis)
3. set the correct path for the *.html templte file
4. set the correct output path that is the same used in item 1 shown above.

Before we begin, set the paths to on your local paths:

In [1]:
import os
import numpy as np

# Define the output path (you can chagne it to a desired path).
output_path=os.path.join(os.path.expanduser('~'),'data-inspection')

# define the gallery templte file path: 
# you can find it in the llsmvis cold package, change the following path into your local path.
gallery_template_path="/Users/yi10/Desktop/Research/Software/T-llsmvis/llsmvis/extensions/hp3d/galleryhead.html"


## Step 1, import and define tools.

In [2]:
def add_an_element_to_gallery(tstr, pic, show_title=True):
    ls=[]
    ls.append("<div class=\"responsive\">\n")
    ls.append("<div class=\"gallery\">\n")
    if show_title:
        ls.append("    <div class=\"desc\">"+tstr+"</div>\n")
    ls.append("<a target=\"_blank\"\n")
    ls.append("href=\"./hp3d/"+tstr+"/"+pic+"\">\n")
    ls.append("<img src=\"./hp3d/"+tstr+"/"+pic+"\"\n")
    ls.append("          alt=\"test\"\n")
    if pic == "thresholds.png":
        ls.append("           width=\"600\"\n")
    else:
        ls.append("           width=\"200\"\n")

    ls.append("           height=\"200\">\n")
    ls.append("    </a>\n")
    ls.append("  </div>\n")
    ls.append("</div>\n")
    return(ls)


def get_gallery_view_one_element_for_all(filename, content, nuner_of_elemenets_per_row, \
                                         gallery_template_path, celltype):
    tag=0
    outhtmlpath=output_path+"/result_gallery_"+celltype+"_"+filename[:-4]+".html"
    cpstr="scp "+gallery_template_path+" "+outhtmlpath
    os.system(cpstr)

    with open(outhtmlpath, 'a') as f:
        f.write('\n')
        f.write("<h1>HP3D result report </h1>\n")
        f.write("<h2> Condition: "+celltype+" macrophage cells. </h2>\n")
        f.write("<h3> Visualization element file name: "+filename+". </h3>\n")
        f.write("<h3> Content: "+content+". </h3>\n")
        f.write("</div>\n")
        f.write("<div style=\"padding:6px;\">\n")
        f.write("<div class=\"clearfix\"></div>\n")
        if celltype=='naive':
            dayslist=days[0:10]
            condition_tags_list=condition_tags[0:10]
        if celltype=='infected':
            dayslist=days[11:]
            condition_tags_list=condition_tags[11:]

        for thisday,condition in zip(dayslist, condition_tags_list):    
            tlist0=[x for x in os.listdir(output_path+'/hp3d') if x.startswith(thisday)]
            cinds=[]
            for t in tlist0:
                ind=t.split('cell')[-1]
                if len(ind)<2:
                    ind='0'+ind
                cinds.append(ind)
            tlist=[x for _, x in sorted(zip(cinds, tlist0))]
            for tstr in tlist: # put in the gif
                tag+=1
                ls=add_an_element_to_gallery(tstr, filename, show_title=True)
                for l in ls:
                    f.write(l)
                f.write("\n")
                if tag//nuner_of_elemenets_per_row==tag/nuner_of_elemenets_per_row: # display 7 elements per row
                    f.write("<div class=\"clearfix\"></div>\n")

        f.write("</body>\n")
        f.write("</html>\n")
        

def get_gallery_view_multiple_elements_for_each_day(filename_list, nuner_of_elemenets_per_row, \
                                         gallery_template_path):
    for thisday,condition in zip(days, condition_tags): # loop over all the days and both conditions
        tlist0=[x for x in os.listdir(output_path+'/hp3d') if x.startswith(thisday)] # get time list (tlist)
        # sort the tlist with direciton of time.
        cinds=[]
        for t in tlist0:
            ind=t.split('cell')[-1]
            if len(ind)<2:
                ind='0'+ind
            cinds.append(ind)
        tlist=[x for _, x in sorted(zip(cinds, tlist0))] 

        # define output html and copy over the html template file.
        outhtmlpath=output_path+"/result_gallery_all_"+thisday+".html"
        cpstr="scp "+gallery_template_path+" "+outhtmlpath
        os.system(cpstr)

        with open(outhtmlpath, 'a') as f:
            f.write('\n')
            f.write("<h2>HP3D result report - "+thisday+" </h2>\n")
            f.write("<h3>"+condition+" </h3>\n")

            f.write("<div style=\"padding:6px;\">\n")
            f.write("  <p>  results for "+thisday+" data </p>\n")
            f.write("</div>\n")
            tin=[]

            N=0
            tall=[]
            tin=[]
            for t in tlist:
                tin.append(t)
                N+=1
                if np.remainder(N,nuner_of_elemenets_per_row) == 0:
                    tall.append(tin)
                    tin=[]
            tall.append(tin)

            f.write("<div class=\"clearfix\"></div>\n")
            for i, tlist_in in enumerate(tall):
                f.write("  <p>  row "+str(i)+" </p>\n")
                for pic in filename_list:
                    for tstr in tlist_in: # put in the gif
                        ls=[]
                        ls.append("<div class=\"responsive\">\n")
                        ls.append("<div class=\"gallery\">\n")
                        if pic == "peripheral_and_internal.gif":
                            ls.append("    <div class=\"desc\">"+tstr+"</div>\n")
                        ls.append("<a target=\"_blank\"\n")
                        ls.append("href=\"./hp3d/"+tstr+"/"+pic+"\">\n")
                        ls.append("<img src=\"./hp3d/"+tstr+"/"+pic+"\"\n")
                        ls.append("          alt=\"test\"\n")
                        if pic == "thresholds.png":
                            ls.append("           width=\"600\"\n")
                        else:
                            ls.append("           width=\"200\"\n")

                        ls.append("           height=\"200\">\n")
                        ls.append("    </a>\n")
                        ls.append("  </div>\n")
                        ls.append("</div>\n")
                        for l in ls:
                            f.write(l)
                        f.write("\n")

                    f.write("<div class=\"clearfix\"></div>\n")



            f.write("</body>\n")
            f.write("</html>\n")

## Step 2, configure parameters.

In [3]:
# create lists of parameters.

# create a list of names of file folders sorted with ascending time
days = np.unique([x.split('-cell')[0] for x in os.listdir(output_path+'/hp3d') if x.startswith('2021')])

# create a list of imaging condision tags. 
# Naive macrohage cells were imaged in the initial 10 days, 
# and infected macrophage cells were imaged in the second 10 days.
condition_tags=['naive macropahge + collagen']*10 + ['Legionella + macropahge + collagen']*10

# define a list of visualization element file names:
piclist=[]
contentlist=[]
piclist.append('peripheral_and_internal.gif')
contentlist.append('Cell peripheral (color coded) and cell internal (gray). ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('peripheral.gif')
contentlist.append('Cell peripheral (color coded). ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('internal.gif')
contentlist.append('Cell internal (color coded). ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('inspect_rgbas_after_cropping_peri2sp_XY_overlay.png')
contentlist.append('Cell peripheral MIP projections in XY plane, 50 time points overlapped. ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('inspect_rgbas_after_cropping_peri2sp_XZ_overlay.png')
contentlist.append('Cell peripheral MIP projections in XZ plane, 50 time points overlapped. ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('inspect_rgbas_after_cropping_peri2sp_YZ_overlay.png')
contentlist.append('Cell peripheral MIP projections in YZ plane, 50 time points overlapped. ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('mass_center_trajecotry.png')
contentlist.append('Nass center trajectory. ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('volume_center_trajectory.png')
contentlist.append('Volume center trajectory. ' +\
                   'Color code: cyan->magenta marks the direction of time.')

piclist.append('MyRep_data_specifics.png')
contentlist.append('Data acquisition specifics. ')

piclist.append('thresholds.png')
contentlist.append('Inspect the segmentation thresholds. ')

piclist.append('rouphness.png')
contentlist.append('Whole cell smoothness evolution curve. ' +\
                   'Color code: cyan->magenta marks the direction of time.')


## Step 3. Generate reports of single type of visualization element.

In [4]:
for ct in ['naive', 'infected']: # loop over 2 conditions
    for ind in np.arange(len(piclist)): # loop over all types of visualization elements from the list
        filename=piclist[ind]
        content=contentlist[ind]
        nuner_of_elemenets_per_row=7
        celltype='infected'
        get_gallery_view_one_element_for_all(filename, content, nuner_of_elemenets_per_row, \
                                             gallery_template_path, celltype=ct)

## Step 4. Generate reports with multiple visualization elements for each experiment day.

In [5]:
nuner_of_elemenets_per_row=7 # define the number of columns. set it to 7.
get_gallery_view_multiple_elements_for_each_day(piclist, nuner_of_elemenets_per_row, \
                                         gallery_template_path)


-- end --